In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import os
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from stop_words import get_stop_words
import re
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, FeatureUnion

from catboost import CatBoostClassifier, Pool
#from sentence_transformers import SentenceTransformer
from typing import Optional, List, Dict

import torch
from numpy import asarray
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tqdm.notebook import tqdm

#from model import ModelForClassification
from gensim.models import FastText


from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix
)


ru_stop = set(get_stop_words("ru"))


In [2]:
train = pd.read_csv('/Users/davidmarshalov/Desktop/Мага/ML/nlp/train.csv (1).csv')
test = pd.read_csv('/Users/davidmarshalov/Desktop/Мага/ML/nlp/test.csv (1).csv')
submission = pd.read_csv('/Users/davidmarshalov/Desktop/Мага/ML/nlp/sample_submission.csv (1).csv')

In [3]:
le = LabelEncoder()

train.rate = le.fit_transform(train.rate)

In [14]:
submission

,index,rate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
12162,12162,1
12163,12163,1
12164,12164,1
12165,12165,1


In [ ]:
import re
import numpy as np
import pandas as pd
from typing import List
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin

# ===== 1) Тональность RU: попытка использовать dostoevsky, иначе fallback =====
_dosto_model = None
def _load_dostoevsky():
    global _dosto_model
    if _dosto_model is not None:
        return _dosto_model
    try:
        from dostoevsky.tokenization import RegexTokenizer
        from dostoevsky.models import FastTextSocialNetworkModel
        tokenizer = RegexTokenizer()
        _dosto_model = FastTextSocialNetworkModel(tokenizer=tokenizer)  # скачает веса при первом запуске
    except Exception:
        _dosto_model = None
    return _dosto_model

# очень маленький словарный fallback (можешь расширить)
_POS = {
"хорошо","отлично","прекрасно","замечательно","великолепно","восхитительно","супер","класс","потрясающе","шикарно",
"удивительно","сильно","сильный","качественный","надёжный","надежный","удобный","комфортный","красивый","стильный",
"аккуратный","чистый","вкусный","сочный","ароматный","свежий","быстрый","оперативный","пунктуальный","вежливый","доброжелательный",
"дружелюбный","отзывчивый","профессиональный","компетентный","умелый","талантливый","мастерский","безупречный","идеальный","ровный",
"стабильный","устойчивый","понятный","ясный","простой","легкий","логичный","удобоваримый","полезный","информативный",
"ценный","выгодный","доступный","недорогой","честный","прозрачный","корректный","терпеливый","внимательный","бережный",
"бережно","бережливый","исправный","рабочий","крепкий","приятный","симпатичный","привлекательный","милый","радостный",
"рад","довольный","счастливый","благодарный","рекомендовать","советовать","хвалить","лайк","топ","лучший",
"лидирующий","первоклассный","премиальный","элитный","крутой","бесподобный","бескомпромиссный","надёжность","надежность","качество",
"комфорт","эстетичный","эстетика","функциональный","универсальный","гибкий","настроемый","быстро","резво","шустро",
"моментально","молниеносно","своевременно","актуальный","релевантный","умный","интеллектуальный","чуткий","тактичный","культурный",
"улыбка","смех","радость","наслаждение","удовольствие","кайф","кайфовый","драйвовый","живой","энергичный",
"позитив","плюс","выигрышный","успешный","успех","результативный","эффективный","эффектный","убедительный","надежно",
"красота","красивость","стильно","элегантный","утонченный","лаконичный","актуально","современно","современный","инновационный",
"прогрессивный","новаторский","модный","эргономичный","предсказуемый","бесшовный","плавный","ровно","ровность","экономичный",
"практичный","уместный","разумный","адекватный","грамотный","чистота","опрятный","опрятность","щедрый","щедро",
"быстрота","чуткость","тактичность","внимание","забота","заботливый","бережность","бережно","деликатный","деликатность",
"команда","командный","поддержка","поддерживать","помогать","помощь","решать","решено","решение","исправлено",
"исправить","улучшено","улучшить","улучшение","апдейт","обновление","апгрейд","рост","развитие","прогресс",
"надёжно","стабильно","ровненько","мощный","сильнейший","богатый","полный","полноценный","адекватность","доброта",
"доброжелательность","уважение","уважительный","корректность","скорость","быстродействие","точный","точность","аккуратность","тщательный",
"тщательно","скрупулёзный","скрупулезный","бережок","фейеричный","огнисто","огонь","огненный","конфетка","конфетный",
"сказочный","легендарный","беспроигрышный","безошибочный","безупречно","идеал","топчик","годный","годно","годность",
"огромный плюс","сверхудобный","сверхбыстрый","сверхкачественный","комплимент","респект","уважуха","браво","благодарю","спасибо"
}

_NEG = {
"плохо","ужасно","отвратительно","мерзко","кошмарно","кошмар","ужас","ужасный","скверно","печально",
"грустно","грусть","тоскливо","тоска","негатив","минус","слабый","слабость","дрянной","паршивый",
"дешёвка","дешевка","кривой","криво","кривизна","косо","косой","кривойство","грязный","грязно",
"грязь","вонючий","вонь","смрад","протухший","просроченный","несвежий","кислый","безвкусный","невкусный",
"жёсткий","жесткий","жевать невозможно","горелый","пересоленный","сырой","непрожаренный","водянистый","пустой","пустышка",
"посредственный","никакой","никакущий","никудышный","ужаснейший","худший","плохой","хуже","дешёвый","дешевый",
"дорогой","переплата","переплачивать","обман","мошенничество","развод","надувательство","подлог","фальшь","лгать",
"ложь","лживый","обманывать","обманутый","хамство","хамский","грубый","грубо","невежливый","неуважительный",
"агрессивный","токсичный","токсичность","ядовитый","яд","боль","болезненный","больно","раздражение","раздражать",
"бесить","бесит","нервировать","нервно","стресс","стрессовый","паника","панический","паниковать","шок",
"шоковый","шокирующий","шокировать","разочарование","разочарован","разочаровать","жалоба","жалкий","жалостный","претензия",
"некомпетентный","непрофессиональный","неграмотный","некорректный","неадекватный","безответственный","безответственность","пофигизм","наплевательский","халатный",
"халатность","небрежный","небрежность","грязнуля","пыльно","пыль","сырость","плесень","ржавчина","ржавый",
"сломанный","поломка","поломанный","дефект","дефектный","брак","бракованный","косяк","косячный","костыль",
"тормозить","тормозной","медленный","медлительный","долго","затянуто","задержка","просрочка","опоздание","опаздывать",
"отмена","срыв","срываться","потеря","потерянный","терять","пропажа","утеря","испортить","испорченный",
"портить","криворукий","криворукость","криворучка","рукожоп","кринж","стыд","стыдно","позор","позорный",
"наглость","наглый","нахамить","хамить","хам","грубить","ругаться","мат","матерный","оскорбление",
"оскорбительный","тупой","глупый","идиотский","идиот","бред","бредовый","ерунда","чушь","чепуха",
"нелогичный","нелепый","сырой код","сырость","недоработка","недоделка","костыльный","костыльно","криво сделано","кривая реализация",
"нестабильный","падает","падение","краш","вылет","вылетать","сбой","ошибка","баг","багованный",
"багнутый","дисконнект","лаг","лагать","тормоза","глючный","глючить","глючит","мертво","мертвый",
"заморозка","замерзать","зависать","завис","перегрев","шумный","шум","скрип","скрипеть","скрежет",
"царапина","царапанный","грязнуть","тонуть","разбитый","разбито","трещина","трещать","раскол","развал",
"разрушение","разрушать","сопротивление","сопротивляться","недоступный","недоступно","неудобный","неудобство","неприятный","неприятность",
"несправедливый","несправедливо","обидно","обида","обидный","некомфортный","некачественный","плоховатый","сомнительный","сомнение"
}


def _sentiment_score_fallback(texts: List[str]) -> np.ndarray:
    scores = []
    for s in texts:
        toks = re.findall(r"\w+", s.lower())
        p = sum(t in _POS for t in toks)
        n = sum(t in _NEG for t in toks)
        # нормируем в [-1,1]
        score = 0.0 if (p+n)==0 else (p - n) / (p + n)
        scores.append(score)
    return np.asarray(scores, dtype=np.float32)

def _sentiment_score_dostoevsky(texts: List[str]) -> np.ndarray:
    model = _load_dostoevsky()
    if model is None:
        return _sentiment_score_fallback(texts)
    # Модель возвращает вероятности классов: negative/neutral/positive/speech/skip
    preds = model.predict(texts, k=3)
    out = []
    for d in preds:
        pos = float(d.get("positive", 0.0))
        neg = float(d.get("negative", 0.0))
        # сводим к числу [-1,1]
        out.append(pos - neg)
    return np.asarray(out, dtype=np.float32)

# Трансформер мета-признаков (тональность + длина)
class MetaFeaturesRU(BaseEstimator, TransformerMixin):
    def __init__(self, use_dostoevsky: bool = True):
        self.use_dostoevsky = use_dostoevsky

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = pd.Series(X).fillna("").astype(str).tolist()
        # длина (лог-масштаб)
        lengths = np.log1p([len(t) for t in texts]).astype(np.float32)
        # тональность
        if self.use_dostoevsky:
            sent = _sentiment_score_dostoevsky(texts)
        else:
            sent = _sentiment_score_fallback(texts)
        feats = np.c_[lengths, sent].astype(np.float32)  # shape [n,2]
        return sparse.csr_matrix(feats)


TF-IDF pipeline

In [ ]:
try:
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    def normalize_ru(s: str) -> str:
        s = str(s).lower()
        s = re.sub(r"[^\w\s]+", " ", s)
        toks = re.findall(r"\w+", s)
        toks = [morph.parse(t)[0].normal_form for t in toks]  # леммы
        if ru_stop:
            toks = [t for t in toks if t not in ru_stop and len(t) > 1]
        return " ".join(toks).strip()
except Exception:
    def normalize_ru(s: str) -> str:
        s = str(s).lower()
        s = re.sub(r"[^\w\s]+", " ", s)
        toks = re.findall(r"\w+", s)
        if ru_stop:
            toks = [t for t in toks if t not in ru_stop and len(t) > 1]
        return " ".join(toks).strip()

# функция-обёртка для Pipeline
def normalize_series(x):
    # x может быть списком/серией строк; возвращаем список строк
    return pd.Series(x).fillna("").astype(str).apply(normalize_ru).tolist()

weightsTFIDF = {0:1.5, 1:3.0, 2:1.2, 3:1.0, 4:0.8}

# pipe = Pipeline([
#     ("normalize", FunctionTransformer(normalize_series, validate=False)),
#     ("features", FeatureUnion([
#         ("char", TfidfVectorizer(
#             analyzer="char_wb",
#             ngram_range=(3, 9),
#             max_features=400000,
#             lowercase=True,
#             dtype=np.float32,
#         )),
#         ("word", TfidfVectorizer(
#             analyzer="word",
#             ngram_range=(1, 4),
#             max_features=300000,
#             lowercase=True,
#             stop_words=list(ru_stop),
#             dtype=np.float32,
#         )),
#     ])),
#     ("clf", LogisticRegression(
#         C=0.5,
#         solver="lbfgs",
#         multi_class="multinomial",
#         class_weight=weightsTFIDF,
#         max_iter=2000,
#         n_jobs=-1,
#         random_state=42,
#     ))
# ])

pipe = Pipeline([
    ("normalize", FunctionTransformer(normalize_series, validate=False)),
    ("features", FeatureUnion([
        ("char", TfidfVectorizer(
            analyzer="char_wb",
            ngram_range=(2, 9),
            max_features=600_000,
            lowercase=True,
            dtype=np.float32,
        )),
        ("word", TfidfVectorizer(
            analyzer="word",
            ngram_range=(1, 5),
            max_features=300_000,
            lowercase=True,
            stop_words=list(ru_stop),
            dtype=np.float32,
        )),
        ("meta", MetaFeaturesRU(use_dostoevsky=True)),  
    ])),
    ("clf", LogisticRegression(
        C=0.5,
        solver="lbfgs",
        multi_class="multinomial",
        class_weight=weightsTFIDF,
        max_iter=2000,
        n_jobs=-1,
        random_state=42,
    ))
])


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pymorphy2/analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
#X_val_text[X_val_text['rateTrue'] != X_val_text['ratePred']]['rateTrue'].value_counts()
#errs = X_val_text[X_val_text["rateTrue"] != X_val_text["ratePred"]]
#модель нечетко привдит границы между соседними рейтингами- дальних промахов мало
#f1 низкий из за классов 1 и 2 так как они лексически похожи на соседей (0 и 3)
#errs.groupby(["rateTrue", "ratePred"]).size().sort_values(ascending=False).reset_index(name="count")

In [ ]:
# # Если использовал LabelEncoder для y:
# # y_val_decoded = le.inverse_transform(y_val)
# # preds_decoded  = le.inverse_transform(preds)
# # Ниже метрики можно считать и по закодированным, и по декодированным — без разницы.
# y_true = y_val
# y_pred = preds

# print("Accuracy:", accuracy_score(y_true, y_pred)) #
# print("F1 macro:", f1_score(y_true, y_pred, average="macro")) # среднее f1 по все класссам с равным весом
# print("F1 weighted:", f1_score(y_true, y_pred, average="weighted")) #среднее F1 с весами по размеру классов.
# print("F1 micro:", f1_score(y_true, y_pred, average="micro")) #

# print("Precision macro:", precision_score(y_true, y_pred, average="macro"))
# print("Recall macro:", recall_score(y_true, y_pred, average="macro"))

# print("\nClassification report (по классам):")
# print(classification_report(y_true, y_pred, digits=4))

# cm = confusion_matrix(y_true, y_pred)
# cm_df = pd.DataFrame(cm, index=sorted(np.unique(y_true)), columns=sorted(np.unique(y_true)))
# print("\nConfusion matrix:")
# print(cm_df)

Accuracy: 0.6312544950169526
F1 macro: 0.45446213750553366
F1 weighted: 0.6150116661947508
F1 micro: 0.6312544950169526
Precision macro: 0.4660275427159302
Recall macro: 0.45124378469067095

Classification report (по классам):
              precision    recall  f1-score   support

           0     0.5297    0.5278    0.5287       828
           1     0.1757    0.2095    0.1911       482
           2     0.4096    0.3216    0.3603      1225
           3     0.4458    0.3191    0.3719      1984
           4     0.7695    0.8782    0.8202      5214

    accuracy                         0.6313      9733
   macro avg     0.4660    0.4512    0.4545      9733
weighted avg     0.6084    0.6313    0.6150      9733


Confusion matrix:
     0    1    2    3     4
0  437  162  108   44    77
1  145  101  137   38    61
2  149  182  394  252   248
3   42   95  228  633   986
4   52   35   95  453  4579


In [6]:
#pipe.fit(train["text"], train["rate"])

# ===== ПРЕДСКАЗАНИЯ =====
# для валидации: preds = pipe.predict(val_text)
# для теста:
# test_pred_codes = pipe.predict(test["text"])
# test_pred_labels = le.inverse_transform(test_pred_codes)

# Predict probabilities
#preds_proba = logreg.predict_proba(X_test)
# Get classes
#preds = np.argmax(preds_proba, axis=1)

# sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
# sample_submission["rate"] = preds
# sample_submission.rate = le.inverse_transform(sample_submission.rate)
# sample_submission.head()

fasttext

text                             
      count unique              top freq
rate                                    
0      4138   4130           Грязно    3
1      2410   2407           Дорого    2
2      6126   6070        Нормально    7
3      9922   9763             Норм   14
4     26069  24804  Хороший магазин  107

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
_token_re = re.compile(r"\w+", flags=re.UNICODE)

def tokenize_series(x: List[str]) -> List[List[str]]:
    return pd.Series(x).fillna("").astype(str).str.lower().apply(lambda s: _token_re.findall(s)).tolist()

class FastTextVectorizer(BaseEstimator, TransformerMixin):
    """
    sklearn-совместимый трансформер:
      fit  -> обучает FastText на токенах (или загружает предобученную модель)
      transform -> усредняет эмбеддинги слов в эмбеддинг текста
    """
    def __init__(self,
                 vector_size: int = 400,
                 window: int = 6,
                 min_count: int = 2,
                 sg: int = 1,
                 epochs: int = 10,
                 workers: int = 4,
                 seed: int = 42,
                 pretrained_path: Optional[str] = None, 
                 l2_normalize: bool = True):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.sg = sg
        self.epochs = epochs
        self.workers = workers
        self.seed = seed
        self.pretrained_path = pretrained_path
        self.l2_normalize = l2_normalize
        self._ft = None  # модель FastText (gensim)

    def fit(self, X: List[List[str]], y=None):
        if self.pretrained_path:
            raise NotImplementedError("Загрузка .bin fastText покажу отдельно при необходимости.")
        else:
            self._ft = FastText(
                sentences=X,
                vector_size=self.vector_size,
                window=self.window,
                min_count=self.min_count,
                sg=self.sg,
                epochs=self.epochs,
                workers=self.workers,
                seed=self.seed
            )
        return self

    def _sent_embed(self, tokens: List[str]) -> np.ndarray:
        if not tokens:
            return np.zeros(self.vector_size, dtype=np.float32)
        vecs = []
        for w in tokens:
            if w in self._ft.wv:
                vecs.append(self._ft.wv[w])
        if not vecs:
            return np.zeros(self.vector_size, dtype=np.float32)
        v = np.mean(vecs, axis=0).astype(np.float32)
        if self.l2_normalize:
            n = np.linalg.norm(v)
            if n > 0:
                v = v / n
        return v

    def transform(self, X: List[List[str]]):
        embs = np.vstack([self._sent_embed(toks) for toks in X]).astype(np.float32)
        return embs

weightsFT = {0:1.5, 1:3.0, 2:1.2, 3:1.0, 4:0.8}

pipe_fasttext = Pipeline([
    ("tokenize", FunctionTransformer(tokenize_series, validate=False)),
    ("ft", FastTextVectorizer(
        vector_size=500, window=5, min_count=2, sg=1,
        epochs=10, workers=4, seed=42, l2_normalize=True
    )),
    ("clf", LogisticRegression(
        C=0.5,
        solver="lbfgs",
        multi_class="multinomial",
        class_weight=weightsFT,
        max_iter=5000,
        n_jobs=-1,
        random_state=42
    ))
])


In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    train["text"].fillna(""),
    train["rate"],
    test_size=0.1,
    stratify=train["rate"],
    random_state=42
)


In [8]:
pipe.fit(train["text"].fillna(""), train["rate"])
#pipe.fit(X_train, y_train)
#pipe_fasttext.fit(X_train, y_train)
pipe_fasttext.fit(train["text"].fillna(""), train["rate"])


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by z

Pipeline(steps=[('tokenize',
                 FunctionTransformer(func=<function tokenize_series at 0x1698b7f60>)),
                ('ft', FastTextVectorizer(vector_size=500, window=5)),
                ('clf',
                 LogisticRegression(C=0.5,
                                    class_weight={0: 1.5, 1: 3.0, 2: 1.2,
                                                  3: 1.0, 4: 0.8},
                                    max_iter=5000, multi_class='multinomial',
                                    n_jobs=-1, random_state=42))])

In [ ]:
preds_tfidf = pipe.predict(X_val)
preds_ft    = pipe_fasttext.predict(X_val)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
# вероятности для усреднения 
proba_tfidf = pipe.predict_proba(X_val)
proba_ft    = pipe_fasttext.predict_proba(X_val)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix

# 1 Поиск лучшего глобального веса w в [0,1]
def search_best_weight(y_true, proba1, proba2, grid=101):
    best = {"w": None, "f1_micro": -1.0, "acc": -1.0}
    for w in np.linspace(0.0, 1.0, grid):
        proba_ens = w * proba1 + (1.0 - w) * proba2
        y_pred = proba_ens.argmax(axis=1)
        f1m = f1_score(y_true, y_pred, average="micro")
        acc = accuracy_score(y_true, y_pred)
        if f1m > best["f1_micro"] or (f1m == best["f1_micro"] and acc > best["acc"]):
            best.update({"w": w, "f1_micro": f1m, "acc": acc})
    return best

# 2 «Порог уверенности»: берём предсказание модели с большей top-probability,
#    но добавляем порог-маржу m: если (p1_max - p2_max) >= m — берём модель 1, иначе модель 2.
def search_best_margin(y_true, proba1, proba2, grid=91):
    p1_max = proba1.max(axis=1)
    p2_max = proba2.max(axis=1)
    y1 = proba1.argmax(axis=1)
    y2 = proba2.argmax(axis=1)

    best = {"m": None, "f1_micro": -1.0, "acc": -1.0}
    for m in np.linspace(0.0, 0.9, grid):
        use_1 = (p1_max - p2_max) >= m
        y_pred = np.where(use_1, y1, y2)
        f1m = f1_score(y_true, y_pred, average="micro")
        acc = accuracy_score(y_true, y_pred)
        if f1m > best["f1_micro"] or (f1m == best["f1_micro"] and acc > best["acc"]):
            best.update({"m": m, "f1_micro": f1m, "acc": acc})
    return best

# 3 Двухпороговое правило (опционально): если модель 1 уверена ≥ τ1 и её top-prob ≥ модели 2 → берём 1,
#    иначе если модель 2 уверена ≥ τ2 и её top-prob ≥ модели 1 → берём 2,
#    иначе — усредняем с весом w* (из шага 1).
def search_best_two_thresholds(y_true, proba1, proba2, w_star, grid_t=25):
    t_vals = np.linspace(0.50, 0.99, grid_t)
    p1_max = proba1.max(axis=1); y1 = proba1.argmax(axis=1)
    p2_max = proba2.max(axis=1); y2 = proba2.argmax(axis=1)
    proba_w = w_star * proba1 + (1 - w_star) * proba2
    yw = proba_w.argmax(axis=1)

    best = {"t1": None, "t2": None, "f1_micro": -1.0, "acc": -1.0}
    for t1 in t_vals:
        for t2 in t_vals:
            pick1 = (p1_max >= t1) & (p1_max >= p2_max)
            pick2 = (p2_max >= t2) & (p2_max >  p1_max)
            # если ни один не «достаточно уверен» — fallback к взвешенному ансамблю
            y_pred = np.where(pick1, y1, np.where(pick2, y2, yw))
            f1m = f1_score(y_true, y_pred, average="micro")
            acc = accuracy_score(y_true, y_pred)
            if f1m > best["f1_micro"] or (f1m == best["f1_micro"] and acc > best["acc"]):
                best.update({"t1": t1, "t2": t2, "f1_micro": f1m, "acc": acc})
    return best

In [ ]:
y_true = y_val
best_w = search_best_weight(y_true, proba_tfidf, proba_ft)
print(f"[Глобальный вес] лучший w={best_w['w']:.3f} | F1_micro={best_w['f1_micro']:.4f} | Acc={best_w['acc']:.4f}")

best_m = search_best_margin(y_true, proba_tfidf, proba_ft)
print(f"[Маржа уверенности] лучший m={best_m['m']:.3f} | F1_micro={best_m['f1_micro']:.4f} | Acc={best_m['acc']:.4f}")

best_t = search_best_two_thresholds(y_true, proba_tfidf, proba_ft, best_w["w"])
print(f"[Два порога] t1={best_t['t1']:.3f}, t2={best_t['t2']:.3f} | F1_micro={best_t['f1_micro']:.4f} | Acc={best_t['acc']:.4f}")

candidates = [
    ("weight", best_w["f1_micro"]),
    ("margin", best_m["f1_micro"]),
    ("two_thresholds", best_t["f1_micro"]),
]
strategy = max(candidates, key=lambda x: x[1])[0]
print("Лучшая стратегия:", strategy)

# финальные предсказания по лучшей стратегии
if strategy == "weight":
    proba_ens = best_w["w"] * proba_tfidf + (1 - best_w["w"]) * proba_ft
    y_pred = proba_ens.argmax(axis=1)

elif strategy == "margin":
    p1_max = proba_tfidf.max(axis=1); y1 = proba_tfidf.argmax(axis=1)
    p2_max = proba_ft.max(axis=1);    y2 = proba_ft.argmax(axis=1)
    use_1 = (p1_max - p2_max) >= best_m["m"]
    y_pred = np.where(use_1, y1, y2)

else:  # "two_thresholds"
    t1, t2 = best_t["t1"], best_t["t2"]
    p1_max = proba_tfidf.max(axis=1); y1 = proba_tfidf.argmax(axis=1)
    p2_max = proba_ft.max(axis=1);    y2 = proba_ft.argmax(axis=1)
    proba_w = best_w["w"] * proba_tfidf + (1 - best_w["w"]) * proba_ft
    yw = proba_w.argmax(axis=1)
    pick1 = (p1_max >= t1) & (p1_max >= p2_max)
    pick2 = (p2_max >= t2) & (p2_max >  p1_max)
    y_pred = np.where(pick1, y1, np.where(pick2, y2, yw))

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 micro:", f1_score(y_true, y_pred, average="micro"))
print("F1 macro:", f1_score(y_true, y_pred, average="macro"))
print("F1 weighted:", f1_score(y_true, y_pred, average="weighted"))
print("Precision macro:", precision_score(y_true, y_pred, average="macro"))
print("Recall macro:", recall_score(y_true, y_pred, average="macro"))

print("\nClassification report (по классам):")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=sorted(np.unique(y_true)), columns=sorted(np.unique(y_true)))
print("\nConfusion matrix:")
print(cm_df)


[Глобальный вес] лучший w=0.530 | F1_micro=0.6482 | Acc=0.6482
[Маржа уверенности] лучший m=0.000 | F1_micro=0.6431 | Acc=0.6431
[Два порога] t1=0.520, t2=0.520 | F1_micro=0.6489 | Acc=0.6489
Лучшая стратегия: two_thresholds
Accuracy: 0.6488596671460859
F1 micro: 0.6488596671460859
F1 macro: 0.4776969966445524
F1 weighted: 0.6338023214772012
Precision macro: 0.48841134915599066
Recall macro: 0.4754335053129563

Classification report (по классам):
              precision    recall  f1-score   support

           0     0.5561    0.5870    0.5711       414
           1     0.1893    0.2199    0.2035       241
           2     0.4478    0.3361    0.3840       613
           3     0.4650    0.3478    0.3979       992
           4     0.7839    0.8865    0.8320      2607

    accuracy                         0.6489      4867
   macro avg     0.4884    0.4754    0.4777      4867
weighted avg     0.6278    0.6489    0.6338      4867


Confusion matrix:
     0   1    2    3     4
0  243  73   4

In [ ]:

TEST_PATH  = "/Users/davidmarshalov/Desktop/Мага/ML/nlp/test.csv (1).csv"
SAMPLE_SUB_PATH = "/Users/davidmarshalov/Desktop/Мага/ML/nlp/sample_submission.csv (1).csv"
OUTPUT_PATH = "/Users/davidmarshalov/Desktop/Мага/ML/nlp/submission_ens_two_thresholds.csv"

test = pd.read_csv(TEST_PATH)
submission = pd.read_csv(SAMPLE_SUB_PATH)  

TEXT_COL = "text"         
X_test = test[TEXT_COL].astype(str).values



proba_tfidf = pipe.predict_proba(X_test)
proba_ft    = pipe_fasttext.predict_proba(X_test)  

w_star = 1.0
proba_ens = w_star * proba_tfidf + (1 - w_star) * proba_ft 
y_pred_enc = proba_ens.argmax(axis=1).astype(int)

y_pred = le.inverse_transform(y_pred_enc)

# Требуются: pipe, pipe_fasttext и le (LabelEncoder, fit на train.rate)
# proba_tfidf = pipe.predict_proba(X_test)
# proba_ft    = pipe_fasttext.predict_proba(X_test)

#  Лучшая стратегия: two_thresholds + fallback на взвешивание
# t1, t2, w_star = 0.500, 0.561, 0.610

# p1_max = proba_tfidf.max(axis=1); y1 = proba_tfidf.argmax(axis=1)
# p2_max = proba_ft.max(axis=1);    y2 = proba_ft.argmax(axis=1)
# proba_w = w_star * proba_tfidf + (1 - w_star) * proba_ft
# y_w = proba_w.argmax(axis=1)

# pick1 = (p1_max >= t1) & (p1_max >= p2_max)
# pick2 = (p2_max >= t2) & (p2_max >  p1_max)
# y_pred_enc = np.where(pick1, y1, np.where(pick2, y2, y_w)).astype(int)

# y_pred = le.inverse_transform(y_pred_enc)

if len(submission) != len(y_pred):
    raise ValueError(f"Размеры не совпали: submission={len(submission)} vs preds={len(y_pred)}")

sub = pd.DataFrame({
    "index": submission["index"].values,   # индекс из sample
    "rate":  y_pred                        # исходные метки после inverse_transform
})

if sub["rate"].dtype == object:
    try:
        sub["rate"] = sub["rate"].astype(int)
    except Exception:
        pass

sub = sub.loc[:, ["index", "rate"]]
sub = sub[~sub["index"].duplicated(keep="first")]

# === 6) Сохранение ===
sub.to_csv(OUTPUT_PATH, index=False)
print("Saved ->", OUTPUT_PATH)
print(sub.head())

Saved -> /Users/davidmarshalov/Desktop/Мага/ML/nlp/submission_ens_two_thresholds.csv
   index  rate
0      0     5
1      1     5
2      2     5
3      3     4
4      4     1


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
w1, w2 = 0.5, 0.5
proba_ens = w1 * proba_tfidf + w2 * proba_ft
preds_ens = proba_ens.argmax(axis=1)

In [17]:
y_true = y_val
y_pred = preds_ens

print("Accuracy:", accuracy_score(y_true, y_pred)) #
print("F1 macro:", f1_score(y_true, y_pred, average="macro")) # среднее f1 по все класссам с равным весом
print("F1 weighted:", f1_score(y_true, y_pred, average="weighted")) #среднее F1 с весами по размеру классов.
print("F1 micro:", f1_score(y_true, y_pred, average="micro")) #

print("Precision macro:", precision_score(y_true, y_pred, average="macro"))
print("Recall macro:", recall_score(y_true, y_pred, average="macro"))

print("\nClassification report (по классам):")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=sorted(np.unique(y_true)), columns=sorted(np.unique(y_true)))
print("\nConfusion matrix:")
print(cm_df)

Accuracy: 0.6451612903225806
F1 macro: 0.4756466106825578
F1 weighted: 0.6323701476255655
F1 micro: 0.6451612903225806
Precision macro: 0.48519454083196834
Recall macro: 0.4741894207467429

Classification report (по классам):
              precision    recall  f1-score   support

           0     0.5440    0.5676    0.5556       414
           1     0.1894    0.2365    0.2103       241
           2     0.4464    0.3328    0.3813       613
           3     0.4583    0.3548    0.4000       992
           4     0.7879    0.8792    0.8310      2607

    accuracy                         0.6452      4867
   macro avg     0.4852    0.4742    0.4756      4867
weighted avg     0.6273    0.6452    0.6324      4867


Confusion matrix:
     0   1    2    3     4
0  235  82   45   24    28
1   72  57   69   22    21
2   64  90  204  150   105
3   26  48  103  352   463
4   35  24   36  220  2292
